In [1]:
#import packages
import numpy as np
import pandas as pd
import tensorflow as tf
from scipy import stats
from tensorflow.keras import activations, backend
import matplotlib.pyplot as plt
import tensorflow.keras as keras
from tensorflow.keras import backend as K
from tensorflow.keras import Model, Input
from keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers import concatenate
from sklearn.linear_model import LogisticRegression
from sklearn.isotonic import IsotonicRegression
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.model_selection import train_test_split
from imblearn import over_sampling
import random
from numpy.random import seed
from sklearn.calibration import calibration_curve
import shap

In [2]:
#the proposed model
seed(100)
def precision(y_true, y_pred):
    # Calculates the precision
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
def recall(y_true, y_pred):
    # Calculates the recall
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall
def build_model(lstm_unit):
    time_input = keras.Input(shape=(200,94), name='time_input')
    time_step_input=keras.Input(shape=(200,1), name='time_step_input')
    time_step_output=layers.Dense(1,use_bias=True)(time_step_input/180)
    time_step_output=1-tf.keras.activations.tanh(tf.math.pow(time_step_output,2))
    time_output = layers.Attention()([time_input,time_input])
    time_output=concatenate([time_input,time_step_output],axis=-1)
    lstm_output2 = layers.LSTM(lstm_unit)(time_output)
    lstm_output2=keras.backend.expand_dims(lstm_output2)
    lstm_output_time, weights1 = layers.MultiHeadAttention(num_heads=2, key_dim=2,attention_axes=(1))(lstm_output2, time_step_output,return_attention_scores=True)
    lstm_output_time=keras.backend.squeeze(lstm_output_time,axis=-1)
    lstm_output = layers.Dense(32, activation="relu")(lstm_output_time)
    lstm_output = Dropout(0.2)(lstm_output)
    preds = layers.Dense(1, activation="sigmoid")(lstm_output)
    model = Model(inputs=[time_input,time_step_input], outputs=preds)
    nadam = keras.optimizers.Nadam()
    model.compile(optimizer=nadam, loss='binary_crossentropy', metrics=['accuracy',recall, precision])
    return model

In [1]:
#model training, calibration and prediction
def model_performance(modelname,task,epoch,unit):
    global traindata
    global predictiontask
    global trainsize
    global testsize
    global calibrationsize
    if task=='onset6':
        traindata=labdata_onset6
    if task=='onset12':
        traindata=labdata_onset12
    if task=='onset24':
        traindata=labdata_onset24
    if task=='onset48':
        traindata=labdata_onset48
    if task=='onset72':
        traindata=labdata_onset72
    if task=='onset168':
        traindata=labdata_onset168
    if task=='surgery0':
        traindata=labdata_surgery0
    if task=='surgery6':
        traindata=labdata_surgery6
    if task=='surgery12':
        traindata=labdata_surgery12
    if task=='surgery24':
        traindata=labdata_surgery24
    if task=='surgery48':
        traindata=labdata_surgery48
    if task=='surgery72':
        traindata=labdata_surgery72
    if task=='random6' or task=='random12' or task=='random24' or task=='random48' or task=='random72' or task=='random168':
        traindata=labdata_random
    if task=='onset6' or task=='onset12' or task=='onset24' or task=='onset48' or task=='onset72' or task=='onset168':
        predictiontask='anyAKI'
    if task=='surgery0' or task=='surgery6' or task=='surgery12' or task=='surgery24' or task=='surgery48' or task=='surgery72':
        predictiontask='anyAKI'
    if task=='random6':
        predictiontask='aki0_6'
    if task=='random12':
        predictiontask='aki0_12'
    if task=='random24':
        predictiontask='aki0_24'
    if task=='random48':
        predictiontask='aki0_48'
    if task=='random72':
        predictiontask='aki0_72'
    if task=='random168':
        predictiontask='aki0_168'
    a=CHDdata_AKI_prediction.drop(['aki0_6', 'aki0_12', 'aki0_24', 'aki0_48',
       'aki0_72', 'aki0_168','anyAKI'],axis=1)
    b=CHDdata_AKI_prediction[[predictiontask]]
    a_train, a_testp, b_train, b_testp = train_test_split(a, b, test_size=0.4,random_state=150)
    ros = over_sampling.RandomOverSampler(random_state=1)
    a_train_ros, b_train_ros = ros.fit_resample(a_train, b_train)
    a_test_ros, b_test_ros=ros.fit_resample(a_testp, b_testp)
    list1=a_train_ros.index.to_list()
    random.shuffle(list1)
    a_train_ros1=a_train_ros.loc[list1,:]
    b_train_ros1=b_train_ros.loc[list1,:]
    list2=a_test_ros.index.to_list()
    random.shuffle(list2)
    a_test_ros1=a_test_ros.loc[list2,:]
    b_test_ros1=b_test_ros.loc[list2,:]
    trainsize=a_train_ros1.shape[0]    
    j=0
    a_train1=np.zeros((trainsize,200,94))
    a_train2=np.zeros((trainsize,200,1))
    basefile='G:/AKIprediction/SHAP_SMOTE_result/'
    for i in a_train_ros1.index.to_list():
        Id=a_train_ros1.loc[i,'ID']
        a_train1[j,:80,:8]=vital_sign_data[Id,:1,:]
        a_train1[j,180:,:8]=vital_sign_data[Id,99:,:]
        a_train1[j,80:180,:8]=vital_sign_data[Id,:,:]
        a_train2[j,:80,:]=traindata[Id,:80,42:]
        a_train2[j,180:,:]=traindata[Id,80:,42:]
        a_train2[j,80:130,:]=traindata[Id,80,42:]+1
        a_train2[j,130:180,:]=traindata[Id,80,42:]+2
        a_train1[j,:80,8:50]=traindata[Id,:80,:42]
        a_train1[j,180:,8:50]=traindata[Id,80:,:42]
        a_train1[j,80:180,8:50]=traindata[Id,79:80,:42]
        a_train1[j,:,50:]=np.tile(np.array(a_train_ros1.iloc[j,:44]),(200,1))
        j+=1
    model=build_model(unit)
    model.fit({'time_input':a_train1,'time_step_input':a_train2},b_train_ros1, epochs=epoch,batch_size=64,
          validation_split=0.15,verbose=2,validation_batch_size=None)
    ourmodelresult=np.zeros((100,13))
    tprs=[]
    precisions=[]
    prob_trues=[]
    re_prob_trues=[]
    mean_fpr = np.linspace(0,1,100)
    mean_recall = np.linspace(0,1,100)
    mean_prob_pred=np.linspace(0,1,20)
    re_mean_prob_pred=np.linspace(0,1,20)
    for s in range(100):
        a_calibration, a_test, b_calibration, b_test = train_test_split(a_test_ros1, b_test_ros1, test_size=0.6)
        calibrationsize=a_calibration.shape[0]
        testsize=a_test.shape[0]
        j=0
        a_calibration1=np.zeros((calibrationsize,200,94))
        a_calibration2=np.zeros((calibrationsize,200,1))
        for i in a_calibration.index.to_list():
            Id=a_calibration.loc[i,'ID']
            a_calibration1[j,:80,:8]=vital_sign_data[Id,:1,:]
            a_calibration1[j,180:,:8]=vital_sign_data[Id,99:,:]
            a_calibration1[j,80:180,:8]=vital_sign_data[Id,:,:]
            a_calibration2[j,:80,:]=traindata[Id,:80,42:]
            a_calibration2[j,180:,:]=traindata[Id,80:,42:]
            a_calibration2[j,80:130,:]=traindata[Id,80,42:]+1
            a_calibration2[j,130:180,:]=traindata[Id,80,42:]+2
            a_calibration1[j,:80,8:50]=traindata[Id,:80,:42]
            a_calibration1[j,180:,8:50]=traindata[Id,80:,:42]
            a_calibration1[j,80:180,8:50]=traindata[Id,79:80,:42]
            a_calibration1[j,:,50:]=np.tile(np.array(a_calibration.iloc[j,:44]),(200,1))
            j+=1
        j=0
        a_test1=np.zeros((testsize,200,94))
        a_test2=np.zeros((testsize,200,1))
        for i in a_test.index.to_list():
            Id=a_test.loc[i,'ID']
            a_test1[j,:80,:8]=vital_sign_data[Id,:1,:]
            a_test1[j,180:,:8]=vital_sign_data[Id,99:,:]
            a_test1[j,80:180,:8]=vital_sign_data[Id,:,:]
            a_test2[j,:80,:]=traindata[Id,:80,42:]
            a_test2[j,180:,:]=traindata[Id,80:,42:]
            a_test2[j,80:130,:]=traindata[Id,80,42:]+1
            a_test2[j,130:180,:]=traindata[Id,80,42:]+2
            a_test1[j,:80,8:50]=traindata[Id,:80,:42]
            a_test1[j,180:,8:50]=traindata[Id,80:,:42]
            a_test1[j,80:180,8:50]=traindata[Id,79:80,:42]
            a_test1[j,:,50:]=np.tile(np.array(a_test.iloc[j,:44]),(200,1))
            j+=1
        yhat=model.predict([a_calibration1,a_calibration2])
        prediction=pd.DataFrame(yhat)
        lrc = LogisticRegression(random_state=0,class_weight='balanced')
        lrc.fit(pd.DataFrame(prediction[0]),b_calibration)
        ir=IsotonicRegression()
        ir.fit(pd.DataFrame(prediction[0]),b_calibration[predictiontask])
        yhattest=model.predict([a_test1,a_test2])
        predictionother=pd.DataFrame(yhattest)
        predictionother[1]=predictionother[0].apply(lambda x:1 if x>=0.5 else 0)
        predictionother1=lrc.predict_proba(pd.DataFrame(predictionother[0]))
        predictionother1=pd.DataFrame(predictionother1)
        predictionother2=ir.predict(pd.DataFrame(predictionother[0]))
        predictionother2=pd.DataFrame(predictionother2)
        predictionother[2]=predictionother1[1]
        predictionother[3]=predictionother1[1].apply(lambda x:1 if x>=0.5 else 0)
        predictionother[4]=predictionother2[0]
        predictionother[4].fillna(0,inplace=True)
        predictionother[5]=predictionother2[0].apply(lambda x:1 if x>=0.5 else 0)
        ourmodelresult[s,0]=metrics.accuracy_score(b_test,predictionother[5])
        ourmodelresult[s,1]=metrics.recall_score(b_test,predictionother[5])
        precision,recall,_ = metrics.precision_recall_curve(b_test,predictionother[4])
        ourmodelresult[s,2]=metrics.auc(recall,precision)
        ourmodelresult[s,3]=metrics.roc_auc_score(b_test,predictionother[4])
        ourmodelresult[s,4]=metrics.roc_auc_score(b_test,predictionother[0])
        ourmodelresult[s,5]=metrics.brier_score_loss(b_test,predictionother[0])
        ourmodelresult[s,6]=metrics.mean_absolute_error(b_test,predictionother[0])
        ourmodelresult[s,7]=metrics.roc_auc_score(b_test,predictionother[2])
        ourmodelresult[s,8]=metrics.brier_score_loss(b_test,predictionother[2])
        ourmodelresult[s,9]=metrics.mean_absolute_error(b_test,predictionother[2])
        ourmodelresult[s,10]=metrics.roc_auc_score(b_test,predictionother[4])
        ourmodelresult[s,11]=metrics.brier_score_loss(b_test,predictionother[4])
        ourmodelresult[s,12]=metrics.mean_absolute_error(b_test,predictionother[4])
        fpr, tpr, _ =metrics.roc_curve(b_test,predictionother[4])
        interp_tpr = np.interp(mean_fpr, fpr, tpr)
        interp_tpr[0] = 0.0
        tprs.append(interp_tpr)
        precision,recall,_ = metrics.precision_recall_curve(b_test,predictionother[4])
        recall=recall[::-1]
        precision=precision[::-1]
        interp_precision = np.interp(mean_recall, recall, precision)
        precisions.append(interp_precision)
        prob_true, prob_pred = calibration_curve(b_test,predictionother[0], n_bins=20,normalize=True)
        interp_prob_true = np.interp(mean_prob_pred, prob_pred, prob_true)
        prob_trues.append(interp_prob_true)
        re_prob_true, re_prob_pred = calibration_curve(b_test,predictionother[4], n_bins=20,normalize=True)
        interp_re_prob_true = np.interp(re_mean_prob_pred, re_prob_pred, re_prob_true)
        re_prob_trues.append(interp_re_prob_true)
    mean_precision = np.mean(precisions, axis=0)
    std_precision = np.std(precisions, axis=0)
    precision_upper = np.minimum(mean_precision + std_precision, 1)
    precision_lower = np.maximum(mean_precision - std_precision, 0)
    pr_curve_onset6=[]
    pr_curve_onset6.append(mean_recall)
    pr_curve_onset6.append(mean_precision)
    pr_curve_onset6.append(precision_upper)
    pr_curve_onset6.append(precision_lower)
    pr_curve_onset6=pd.DataFrame(pr_curve_onset6)
    pr_curve_onset6=pr_curve_onset6.T
    mean_prob_trues = np.mean(prob_trues, axis=0)
    std_prob_trues = np.std(prob_trues, axis=0)
    prob_true_upper = np.minimum(mean_prob_trues + std_prob_trues, 1)
    prob_true_lower = np.maximum(mean_prob_trues - std_prob_trues, 0)
    calibration_curve_onset6=[]
    calibration_curve_onset6.append(mean_prob_pred)
    calibration_curve_onset6.append(mean_prob_trues)
    calibration_curve_onset6.append(prob_true_upper)
    calibration_curve_onset6.append(prob_true_lower)
    calibration_curve_onset6=pd.DataFrame(calibration_curve_onset6)
    calibration_curve_onset6=calibration_curve_onset6.T
    re_mean_prob_trues = np.mean(re_prob_trues, axis=0)
    re_std_prob_trues = np.std(re_prob_trues, axis=0)
    re_prob_true_upper = np.minimum(re_mean_prob_trues + re_std_prob_trues, 1)
    re_prob_true_lower = np.maximum(re_mean_prob_trues - re_std_prob_trues, 0)
    re_calibration_curve_onset6=[]
    re_calibration_curve_onset6.append(re_mean_prob_pred)
    re_calibration_curve_onset6.append(re_mean_prob_trues)
    re_calibration_curve_onset6.append(re_prob_true_upper)
    re_calibration_curve_onset6.append(re_prob_true_lower)
    re_calibration_curve_onset6=pd.DataFrame(re_calibration_curve_onset6)
    re_calibration_curve_onset6=re_calibration_curve_onset6.T
    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    roc_curve_onset6=[]
    roc_curve_onset6.append(mean_fpr)
    roc_curve_onset6.append(mean_tpr)
    roc_curve_onset6.append(tprs_upper)
    roc_curve_onset6.append(tprs_lower)
    roc_curve_onset6=pd.DataFrame(roc_curve_onset6)
    roc_curve_onset6=roc_curve_onset6.T
    performance_metric=np.zeros((13,2))
    performance_metric=pd.DataFrame(performance_metric)
    ci0=stats.t.interval(0.95,99, loc=np.mean(ourmodelresult[:,0]), scale=stats.sem(ourmodelresult[:,0]))
    mean0=np.mean(ourmodelresult[:,0])
    performance_metric.iloc[0,0]='accuracy'
    performance_metric.iloc[0,1]=str(format(mean0,'.3f'))+' ['+str(format(ci0[0],'.3f'))+'-'+str(format(ci0[1],'.3f'))+']'
    ci1=stats.t.interval(0.95,99, loc=np.mean(ourmodelresult[:,1]), scale=stats.sem(ourmodelresult[:,1]))
    mean1=np.mean(ourmodelresult[:,1])
    performance_metric.iloc[1,0]='recall'
    performance_metric.iloc[1,1]=str(format(mean1,'.3f'))+' ['+str(format(ci1[0],'.3f'))+'-'+str(format(ci1[1],'.3f'))+']'
    ci2=stats.t.interval(0.95,99, loc=np.mean(ourmodelresult[:,2]), scale=stats.sem(ourmodelresult[:,2]))
    mean2=np.mean(ourmodelresult[:,2])
    performance_metric.iloc[2,0]='PR AUC'
    performance_metric.iloc[2,1]=str(format(mean2,'.3f'))+' ['+str(format(ci2[0],'.3f'))+'-'+str(format(ci2[1],'.3f'))+']'
    ci3=stats.t.interval(0.95,99, loc=np.mean(ourmodelresult[:,3]), scale=stats.sem(ourmodelresult[:,3]))
    mean3=np.mean(ourmodelresult[:,3])
    performance_metric.iloc[3,0]='ROC AUC'
    performance_metric.iloc[3,1]=str(format(mean3,'.3f'))+' ['+str(format(ci3[0],'.3f'))+'-'+str(format(ci3[1],'.3f'))+']'
    ci4=stats.t.interval(0.95,99, loc=np.mean(ourmodelresult[:,4]), scale=stats.sem(ourmodelresult[:,4]))
    mean4=np.mean(ourmodelresult[:,4])
    performance_metric.iloc[4,0]='uncalibration AUC'
    performance_metric.iloc[4,1]=str(format(mean4,'.3f'))+' ['+str(format(ci4[0],'.3f'))+'-'+str(format(ci4[1],'.3f'))+']'
    ci5=stats.t.interval(0.95,99, loc=np.mean(ourmodelresult[:,5]), scale=stats.sem(ourmodelresult[:,5]))
    mean5=np.mean(ourmodelresult[:,5])
    performance_metric.iloc[5,0]='uncalibration brier'
    performance_metric.iloc[5,1]=str(format(mean5,'.3f'))+' ['+str(format(ci5[0],'.3f'))+'-'+str(format(ci5[1],'.3f'))+']'
    ci6=stats.t.interval(0.95,99, loc=np.mean(ourmodelresult[:,6]), scale=stats.sem(ourmodelresult[:,6]))
    mean6=np.mean(ourmodelresult[:,6])
    performance_metric.iloc[6,0]='uncalibration mae'
    performance_metric.iloc[6,1]=str(format(mean6,'.3f'))+' ['+str(format(ci6[0],'.3f'))+'-'+str(format(ci6[1],'.3f'))+']'
    ci7=stats.t.interval(0.95,99, loc=np.mean(ourmodelresult[:,7]), scale=stats.sem(ourmodelresult[:,7]))
    mean7=np.mean(ourmodelresult[:,7])
    performance_metric.iloc[7,0]='scaling AUC'
    performance_metric.iloc[7,1]=str(format(mean7,'.3f'))+' ['+str(format(ci7[0],'.3f'))+'-'+str(format(ci7[1],'.3f'))+']'
    ci8=stats.t.interval(0.95,99, loc=np.mean(ourmodelresult[:,8]), scale=stats.sem(ourmodelresult[:,8]))
    mean8=np.mean(ourmodelresult[:,8])
    performance_metric.iloc[8,0]='scaling brier'
    performance_metric.iloc[8,1]=str(format(mean8,'.3f'))+' ['+str(format(ci8[0],'.3f'))+'-'+str(format(ci8[1],'.3f'))+']'
    ci9=stats.t.interval(0.95,99, loc=np.mean(ourmodelresult[:,9]), scale=stats.sem(ourmodelresult[:,9]))
    mean9=np.mean(ourmodelresult[:,9])
    performance_metric.iloc[9,0]='scaling mae'
    performance_metric.iloc[9,1]=str(format(mean9,'.3f'))+' ['+str(format(ci9[0],'.3f'))+'-'+str(format(ci9[1],'.3f'))+']'
    ci10=stats.t.interval(0.95,99, loc=np.mean(ourmodelresult[:,10]), scale=stats.sem(ourmodelresult[:,10]))
    mean10=np.mean(ourmodelresult[:,10])
    performance_metric.iloc[10,0]='isotonic AUC'
    performance_metric.iloc[10,1]=str(format(mean10,'.3f'))+' ['+str(format(ci10[0],'.3f'))+'-'+str(format(ci10[1],'.3f'))+']'
    ci11=stats.t.interval(0.95,99, loc=np.mean(ourmodelresult[:,11]), scale=stats.sem(ourmodelresult[:,11]))
    mean11=np.mean(ourmodelresult[:,11])
    performance_metric.iloc[11,0]='isotonic brier'
    performance_metric.iloc[11,1]=str(format(mean11,'.3f'))+' ['+str(format(ci11[0],'.3f'))+'-'+str(format(ci11[1],'.3f'))+']'
    ci12=stats.t.interval(0.95,99, loc=np.mean(ourmodelresult[:,12]), scale=stats.sem(ourmodelresult[:,12]))
    mean12=np.mean(ourmodelresult[:,12])
    performance_metric.iloc[12,0]='isotonic mae'
    performance_metric.iloc[12,1]=str(format(mean12,'.3f'))+' ['+str(format(ci12[0],'.3f'))+'-'+str(format(ci12[1],'.3f'))+']'
    performance_metric.to_csv(basefile+task+'_result/performance_metric'+task+'_'+modelname+'.csv',index=False,encoding='gbk')
    roc_curve_onset6.columns=['mean_fpr','mean_tpr','tprs_upper','tprs_lower']
    roc_curve_onset6.to_csv(basefile+task+'_result/roc_curve'+task+'_'+modelname+'.csv',index=False,encoding='gbk')
    pr_curve_onset6.columns=['mean_recall','mean_precision','precision_upper','precision_lower']
    pr_curve_onset6.to_csv(basefile+task+'_result/pr_curve'+task+'_'+modelname+'.csv',index=False,encoding='gbk')
    re_calibration_curve_onset6.columns=['mean_prob_pred','mean_prob_trues','prob_true_upper','prob_true_lower']
    re_calibration_curve_onset6.to_csv(basefile+task+'_result/re_calibration_curve'+task+'_'+modelname+'.csv',index=False,encoding='gbk')
    calibration_curve_onset6.columns=['mean_prob_pred','mean_prob_trues','prob_true_upper','prob_true_lower']
    calibration_curve_onset6.to_csv(basefile+task+'_result/calibration_curve'+task+'_'+modelname+'.csv',index=False,encoding='gbk')
    explainer = shap.GradientExplainer(model,[a_train1,a_train2])
    shap_values = explainer.shap_values([a_test1,a_test2])
    shap_value=shap_values[0][0]
    shap_value_time=shap_values[0][1]
    np.save(basefile+task+'_result/shap_value'+task+'_'+modelname+'.npy',shap_value)
    np.save(basefile+task+'_result/shap_value_time'+task+'_'+modelname+'.npy',shap_value_time)
    np.save(basefile+task+'_result/a_test1'+task+'_'+modelname+'.npy',a_test1)
    np.save(basefile+task+'_result/a_test2'+task+'_'+modelname+'.npy',a_test2)